# Raciocínio Baseado em Casos - Artrite

Colocar arquivo excel na pasta dentro do Colab para ser lido pelo código.

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import files

Gestão da base de casos
- carregar a base (arquivo excel)
- adicionar novo caso para a base

In [ ]:
def carregar_base_real():
    """Carrega a base de dados real do arquivo Excel"""
    try:
        # Tenta carregar o arquivo de diferentes formas
        try:
            # Primeiro tenta com sheet_name='Planilha1'
            base_dados = pd.read_excel('Modelo_RBC.xlsx', sheet_name='Planilha1')
        except:
            try:
                # Se falhar, tenta a primeira planilha (índice 0)
                base_dados = pd.read_excel('Modelo_RBC.xlsx', sheet_name=0)
                print("Base carregada da primeira planilha")
            except:
                # Se ainda falhar, tenta sem sheet_name
                base_dados = pd.read_excel('Modelo_RBC.xlsx')
                print("Base carregada (planilha padrão)")

        # Remove a última linha vazia se existir
        base_dados = base_dados.dropna(subset=['Caso'])

        print(f"Base carregada: {len(base_dados)} casos encontrados")
        print(f"Diagnósticos: {base_dados['Diagnostico'].value_counts().to_dict()}")
        return base_dados
    except Exception as e:
        print(f"Erro ao carregar base: {e}")
        print("Por favor, faça upload do arquivo 'Modelo_RBC.xlsx'")
        return None

def salvar_novo_caso(caso_novo, base_dados, resultados, arquivo='Modelo_RBC.xlsx'):
    """Salva um novo caso na base de dados"""
    try:
        # Determina o próximo número do caso
        ultimo_caso = base_dados['Caso'].max()
        novo_id = int(ultimo_caso) + 1
        caso_novo['Caso'] = novo_id # adiciona o ID do caso

        # Pega o diagnóstico do caso mais similar
        if resultados:
            caso_mais_similar = resultados[0][2]  # pega valor do diagnostico
            diagnostico = caso_mais_similar.get('Diagnostico', 'nao') # atribui valor para novo caso
            print(f"    - Diagnóstico atribuído: {diagnostico} (baseado no caso {caso_mais_similar.get('Caso')})")
        else:
            diagnostico = 'nao'
            print("   - Nenhum caso similar encontrado, diagnóstico definido como 'nao'")

        # Converte para o formato da base
        caso_formatado = {}
        for col in base_dados.columns:
            if col in caso_novo:
                caso_formatado[col] = caso_novo[col]
            elif col == 'Diagnostico':
                caso_formatado[col] = diagnostico
            elif col == 'Caso':
                caso_formatado[col] = novo_id
            else:
                caso_formatado[col] = 'nao'

        # Adiciona à base
        novo_dado = pd.DataFrame([caso_formatado])
        df_atualizado = pd.concat([base_dados, novo_dado], ignore_index=True)
        df_atualizado.to_excel(arquivo, index=False) # salva no arquivo

        print(f"\nNovo caso {novo_id} salvo com sucesso na base!")
        return novo_id
    except Exception as e:
        print(f"\nErro ao salvar novo caso: {e}")
        return None

Configuração dos pesos e métricas de similaridade

In [ ]:
PESOS = {
    # Sintomas
    'DL': 1.0,           # Dor Lombar
    'RC': 0.9,           # Rigidez na Coluna
    'DC': 1.0,           # Deformação na Coluna
    'Mob': 0.7,          # Mobilidade
    'DTS': 0.9,          # Dor ao Toque no Sacrolíaco
    'ART': 0.7,          # Artrite
    'RM': 0.6,           # Rigidez Matinal
    'Bur': 0.4,          # Bursite
    'Tof': 1.0,          # Tophi
    'Sin': 0.7,          # Sinovite
    'ATG': 0.6,          # Atralgia
    'NR': 0.3,           # Nódulos Reumatóides
    'HLA-B27': 1.0,      # HLA-B27
    'DJ': 0.3,           # Deformação nas Juntas

    # Exames Clínicos
    'IL': 0.6,           # Inflamação Laboratorial
    'ER': 0.6,           # Evidências Radiológicas
    'TCSE': 0.4          # Tomografia Computadorizada
}

# Matriz de similaridade para atributos categóricos ordinais
MATRIZ_SIMILARIDADE = {
    'ausente': {'ausente': 1.0, 'leve': 0.8, 'moderado': 0.5, 'importante': 0.3, 'muito_importante': 0.0},
    'leve': {'ausente': 0.8, 'leve': 1.0, 'moderado': 0.8, 'importante': 0.5, 'muito_importante': 0.3},
    'moderado': {'ausente': 0.5, 'leve': 0.8, 'moderado': 1.0, 'importante': 0.8, 'muito_importante': 0.5},
    'importante': {'ausente': 0.3, 'leve': 0.5, 'moderado': 0.8, 'importante': 1.0, 'muito_importante': 0.8},
    'muito_importante': {'ausente': 0.0, 'leve': 0.3, 'moderado': 0.5, 'importante': 0.8, 'muito_importante': 1.0}
}

# Dicionário global de descrições dos atributos
DESCRICOES_ATRIBUTOS = {
    'DL': 'Dor Lombar',
    'RC': 'Rigidez na Coluna',
    'DC': 'Deformação na Coluna',
    'Mob': 'Mobilidade',
    'DTS': 'Dor ao Toque Sacrolíaco',
    'ART': 'Artrite',
    'RM': 'Rigidez Matinal',
    'Bur': 'Bursite',
    'Tof': 'Tophi',
    'Sin': 'Sinovite',
    'ATG': 'Atralgia',
    'NR': 'Nódulos Reumatóides',
    'HLA-B27': 'HLA-B27',
    'DJ': 'Deformação nas Juntas',
    'IL': 'Inflamação Laboratorial',
    'ER': 'Evidências Radiológicas',
    'TCSE': 'Tomografia Computadorizada'
}

# Listas globais para categorização
SINTOMAS = ['DL', 'RC', 'DC', 'DTS', 'ART', 'RM', 'Bur', 'Tof', 'Sin', 'ATG', 'NR', 'DJ']
EXAMES = ['Mob', 'IL', 'ER', 'TCSE', 'HLA-B27']

Funções de Similaridade

In [ ]:
def similaridade_nominal(valor1, valor2):
    """Similaridade para atributos nominais (sim/não, normal/limitado)
       Retorna 1 para valores iguais
       Retorna 0 para valores diferentes"""
    if pd.isna(valor1) or pd.isna(valor2):
        return 0.0
    return 1.0 if valor1 == valor2 else 0.0

def similaridade_ordinal(valor1, valor2):
    """Similaridade para atributos ordinais
       Retorna peso da comparação dos atributos"""
    if pd.isna(valor1) or pd.isna(valor2) or valor1 not in MATRIZ_SIMILARIDADE or valor2 not in MATRIZ_SIMILARIDADE[valor1]:
        return 0.0
    return MATRIZ_SIMILARIDADE[valor1][valor2]

def similaridade_numerica(valor1, valor2, faixa_min=0, faixa_max=1):
    """Similaridade para como HLA-B27"""
    if pd.isna(valor1) or pd.isna(valor2):
        return 0.0

    try:
        v1 = float(valor1)
        v2 = float(valor2)
        diferenca = abs(v1 - v2)
        faixa_total = faixa_max - faixa_min
        similaridade = max(0, 1 - (diferenca / faixa_total))
        return similaridade
    except (ValueError, TypeError):
        return 0.0

Calcular Similaridade
- similaridade local: pesos de atributos dos casos
- similaridade global:

In [ ]:
def calcular_similaridade_local(atributo, valor_caso, valor_base):
    """Calcula a similaridade local para um atributo específico"""
    # Se algum valor for None (apenas para valores realmente ausentes/nulos)
    if pd.isna(valor_caso) or pd.isna(valor_base):
        return 0.0

    # Determina o tipo de similaridade baseado no atributo
    if atributo == 'HLA-B27':
        try:
            v1 = float(valor_caso)
            v2 = float(valor_base)
            return similaridade_numerica(v1, v2, 0, 1)
        except (ValueError, TypeError):
            return 0.0

    # Atributos que usam escala ordinal (IL, ER)
    escalas_ordinais = ['ausente', 'leve', 'moderado', 'importante', 'muito_importante']
    if str(valor_caso).lower() in escalas_ordinais and str(valor_base).lower() in escalas_ordinais:
        return similaridade_ordinal(str(valor_caso).lower(), str(valor_base).lower())

    # TCSE com escala específica
    if atributo == 'TCSE':
        escalas_tcse = ['nao', 'leve', 'moderado', 'importante']
        if str(valor_caso).lower() in escalas_tcse and str(valor_base).lower() in escalas_tcse:
            return similaridade_nominal(str(valor_caso).lower(), str(valor_base).lower())

    # Atributos binários (todos os outros)
    escalas_binarias = ['sim', 'nao', 'normal', 'limitado']
    if str(valor_caso).lower() in escalas_binarias and str(valor_base).lower() in escalas_binarias:
        return similaridade_nominal(str(valor_caso).lower(), str(valor_base).lower())

    # Fallback para comparação direta
    return 1.0 if str(valor_caso).lower() == str(valor_base).lower() else 0.0

def similaridade_global(caso_novo, caso_base, pesos):
    """Calcula a similaridade global usando o algoritmo de vizinhança"""
    soma_similaridades = 0
    soma_pesos = 0

    for atributo, peso in pesos.items():
        if atributo in caso_novo and atributo in caso_base:
            similaridade_local = calcular_similaridade_local(
                atributo, # nome do atributo
                caso_novo[atributo], # pega valor do atributo do caso novo
                caso_base[atributo]  # pega valor do atributo do caso da base de dados
            )
            soma_similaridades += similaridade_local * peso
            soma_pesos += peso

    return (soma_similaridades / soma_pesos) * 100 if soma_pesos > 0 else 0

Funções para obter valores para novo caso do usuário

In [ ]:
def mostrar_opcoes_atributo(atributo, tipo):
    """Mostra as opções disponíveis para um atributo"""
    print()
    if tipo == 'binario':
        print("- Opções: sim, não")
    elif tipo == 'mobilidade':
        print("- Opções: normal, limitado")
    elif tipo == 'intensidade':
          print("- Opções: ausente, leve, moderado, importante, muito_importante")
    elif tipo == 'numerico':
        print("- Valor entre 0 e 1")

def obter_entrada_usuario():
    """Obtém os dados do caso novo do usuário"""
    caso_novo = {}
    print("\nInforme os dados do caso para diagnóstico:")

    # Sintomas binários
    for sintoma in SINTOMAS:
        while True:
            mostrar_opcoes_atributo(sintoma, 'binario')
            valor = input(f"{sintoma}: ").strip().lower()
            # Corrige possíveis erros de digitação
            if valor in ['s', 'si', 'sim']:
                valor = 'sim'
            elif valor in ['n', 'no', 'nao', 'não']:
                valor = 'nao'

            # Validação final
            if valor in ['sim', 'nao']:
                caso_novo[sintoma] = valor
                break
            else:
                print("Valor inválido. Tente novamente")

    # Mobilidade
    while True:
        mostrar_opcoes_atributo('Mob', 'mobilidade')
        mob_valor = input("Mob (Mobilidade): ").strip().lower()
        if mob_valor in ['limitada', 'limitado', 'l']:
            mob_valor = 'limitado'
        elif mob_valor in ['normal', 'n', 'norma']:
            mob_valor = 'normal'

        # Validação
        if mob_valor in ['normal', 'limitado']:
            caso_novo['Mob'] = mob_valor
            break
        else:
            print("Valor inválido. Tente novamente")


    # Para IL e ER
    for exame in ['IL', 'ER']:
        while True:
            mostrar_opcoes_atributo(exame, 'intensidade')
            exame_valor = input(f"{exame}: ").strip().lower()
            if exame_valor in ['muito_importante', 'muito', 'mu', 'muito importante', 'mi']:
                exame_valor = 'muito_importante'
            elif exame_valor in ['l', 'leve']:
                exame_valor = 'leve'
            elif exame_valor in ['m', 'moderado', 'moderada', 'mod']:
                exame_valor = 'moderado'
            elif exame_valor in ['i', 'importante', 'imp']:
                exame_valor = 'importante'
            elif exame_valor in ['a', 'ausente', 'aus']:
                exame_valor = 'ausente'

            # Validação
            if exame_valor in ['ausente', 'leve', 'moderado', 'importante', 'muito_importante']:
                caso_novo[exame] = exame_valor
                break
            else:
              print("Valor inválido. Tente novamente")

    # TCSE com opções específicas
    while True:
        print("\n- TCSE - Opções: nao, leve, moderado, importante")
        tcse_valor = input("TCSE: ").strip().lower()
        # Corrige possíveis entradas
        if tcse_valor in ['n', 'nao', 'não']:
            tcse_valor = 'nao'
        elif tcse_valor in ['l', 'leve']:
            tcse_valor = 'leve'
        elif tcse_valor in ['m', 'moderado', 'moderada', 'mod']:
            tcse_valor = 'moderado'
        elif tcse_valor in ['i', 'importante', 'imp']:
            tcse_valor = 'importante'

        # Validação
        if tcse_valor in ['nao', 'leve', 'moderado', 'importante']:
            caso_novo['TCSE'] = tcse_valor
            break
        else:
            print("Valor inválido. Tente novamente")

    # HLA-B27 numérico
    while True:
        try:
            mostrar_opcoes_atributo('HLA-B27', 'numerico')
            valor = float(input("HLA-B27 (0-1): "))
            if 0 <= valor <= 1:
                caso_novo['HLA-B27'] = valor
                break
            else:
                print("Valor deve estar entre 0 e 1")
        except ValueError:
            print("Valor inválido. Tente novamente")

    return caso_novo

Funções para mostrar tabela de resultados e estatísticas calculadas

In [ ]:
def mostrar_resultados(caso_novo, resultados):
    """Exibe os resultados da consulta com TODOS os casos da base"""
    print("\nRESULTADOS DA CONSULTA")
    print("="*120)

    # Cabeçalho da tabela de casos
    print(f"{'Caso':<8} {'Similaridade':<15} {'Diagnóstico':<25} {'DL':<4} {'RC':<4} {'DC':<4} {'Mob':<8} {'DTS':<4} {'IL':<20} {'ER':<20} {'TCSE':<12} {'ART':<4} {'RM':<4} {'Bur':<4} {'Tof':<4} {'Sin':<4} {'ATG':<4} {'NR':<4} {'HLA-B27':<8} {'DJ':<4}")
    print("-" * 160)

    # Mostra dados do caso novo
    print(f"{'Atual':<8} {'---':<15} {'---':<25} "
          f"{caso_novo.get('DL', 'nao'):<4} {caso_novo.get('RC', 'nao'):<4} {caso_novo.get('DC', 'nao'):<4} "
          f"{caso_novo.get('Mob', 'nao'):<8} {caso_novo.get('DTS', 'nao'):<4} {caso_novo.get('IL', 'nao'):<20} "
          f"{caso_novo.get('ER', 'nao'):<20} {caso_novo.get('TCSE', 'nao'):<12} {caso_novo.get('ART', 'nao'):<4} "
          f"{caso_novo.get('RM', 'nao'):<4} {caso_novo.get('Bur', 'nao'):<4} {caso_novo.get('Tof', 'nao'):<4} "
          f"{caso_novo.get('Sin', 'nao'):<4} {caso_novo.get('ATG', 'nao'):<4} {caso_novo.get('NR', 'nao'):<4} "
          f"{caso_novo.get('HLA-B27', 0):<8.1f} {caso_novo.get('DJ', 'nao'):<4}")
    print("-" * 160) # separador entre caso novo e os outros

    # Tabela de resultados completa dos casos da base
    for i, (idx, similaridade, caso) in enumerate(resultados):
        try:
            hla = float(caso.get('HLA-B27', 0)) # converte HLA-B27 para float para evitar erro
        except (ValueError, TypeError):
            hla = 0.0

        print(f"{caso.get('Caso', 'N/A'):<8} {similaridade:<6.1f}%{'':<8} {caso.get('Diagnostico', 'N/A'):<25} "
              f"{caso.get('DL', 'nao'):<4} {caso.get('RC', 'nao'):<4} {caso.get('DC', 'nao'):<4} "
              f"{caso.get('Mob', 'nao'):<8} {caso.get('DTS', 'nao'):<4} {caso.get('IL', 'nao'):<20} "
              f"{caso.get('ER', 'nao'):<20} {caso.get('TCSE', 'nao'):<12} {caso.get('ART', 'nao'):<4} "
              f"{caso.get('RM', 'nao'):<4} {caso.get('Bur', 'nao'):<4} {caso.get('Tof', 'nao'):<4} "
              f"{caso.get('Sin', 'nao'):<4} {caso.get('ATG', 'nao'):<4} {caso.get('NR', 'nao'):<4} "
              f"{hla:<8.1f} {caso.get('DJ', 'nao'):<4}")

    # Estatísticas finais
    print(f"\nESTATÍSTICAS:")

    # Contagem de diagnósticos nos casos mais similares
    top_5 = resultados[:5]
    diagnosticos = {}

    for _, _, caso in top_5:
        diag = caso.get('Diagnostico', 'Desconhecido')
        diagnosticos[diag] = diagnosticos.get(diag, 0) + 1

    print("\n5 casos mais similares:")
    for diag, count in diagnosticos.items():
        porcentagem = (count / len(top_5)) * 100
        print(f"  {diag}: {count} casos ({porcentagem:.1f}%)")

    # Mostra caso mais similar
    caso_mais_similar = top_5[0][2]
    print(f"\nCaso mais similar: {caso_mais_similar.get('Caso')}")
    print(f"  - Diagnóstico: {caso_mais_similar.get('Diagnostico')}")
    print(f"  - Similaridade: {top_5[0][1]:.1f}%")

Alteração das configurações de pesos

In [ ]:
def mostrar_configuracao_pesos():
    """Mostra a configuração atual dos pesos"""
    print("\nConfiguração Atual dos Pesos")

    print(f"\n{'SINTOMAS':<15} {'Peso':<10} {'Descrição'}")
    print("-" * 100)
    for atributo in SINTOMAS:
        if atributo in PESOS:
            descricao = DESCRICOES_ATRIBUTOS.get(atributo, atributo)
            print(f"{atributo:<15} {PESOS[atributo]:<10} {descricao}")

    print(f"\n{'EXAMES':<15} {'Peso':<10} {'Descrição'}")
    print("-" * 50)
    for atributo in EXAMES:
        if atributo in PESOS:
            descricao = DESCRICOES_ATRIBUTOS.get(atributo, atributo)
            print(f"{atributo:<15} {PESOS[atributo]:<10} {descricao}")

    print(f"\nTotal de atributos: {len(PESOS)}")
    print(f"Soma total dos pesos: {sum(PESOS.values()):.1f}")


def modificar_pesos():
    """Permite ao usuário modificar os pesos dos atributos"""
    global PESOS

    while True:
        print("\n\nMODIFICAÇÃO DE PESOS")

        # Mostra lista numerada com descrições completas
        print("-" * 50)
        atributos = list(PESOS.keys())
        for i, atributo in enumerate(atributos, 1):
            descricao = DESCRICOES_ATRIBUTOS.get(atributo, atributo)
            peso_atual = PESOS[atributo]
            print(f"{i:2d} - {atributo:<8} | Peso: {peso_atual:.1f} | {descricao}")
        print("-" * 50)

        print("\n1 - Modificar peso de um atributo")
        print("2 - Restaurar pesos padrão")
        print("3 - Fazer consulta")

        opcao = input("\nEscolha uma opção (1-3): ").strip()

        if opcao == '1':
            try:
                num = int(input(f"\nNúmero do atributo (1-{len(PESOS)}): "))
                if 1 <= num <= len(atributos):
                    atributo = atributos[num-1]
                    descricao = DESCRICOES_ATRIBUTOS.get(atributo, atributo)
                    novo_peso = float(input(f"Novo peso para {atributo} ({descricao}) (0-1): "))

                    if 0 <= novo_peso <= 1:
                        PESOS[atributo] = round(novo_peso, 1)  # Arredonda para 1 casa decimal
                        print(f"{atributo} atualizado para {PESOS[atributo]:.1f}")
                    else:
                        print("Peso deve estar entre 0 e 1")
                else:
                    print("Número inválido")
            except (ValueError, IndexError):
                print("Entrada inválida")

        elif opcao == '2': # pesos padrão
            PESOS.update({
                'DL': 1.0, 'RC': 0.9, 'DC': 1.0, 'Mob': 0.7, 'DTS': 0.9, 'ART': 0.7,
                'RM': 0.6, 'Bur': 0.4, 'Tof': 1.0, 'Sin': 0.7, 'ATG': 0.6, 'NR': 0.3,
                'HLA-B27': 1.0, 'DJ': 0.3, 'IL': 0.6, 'ER': 0.6, 'TCSE': 0.4
            })
            print("Pesos padrão restaurados")

        elif opcao == '3': # volta pro menu principal
            break

        else:
            print("Opção inválida")

Execução do sistema RBC

In [ ]:
def main():
    """Função principal do sistema RBC"""
    # Carrega a base de casos real
    base_casos = carregar_base_real()
    if base_casos is None:
        return

    while True:
        mudar_pesos = input("\nDeseja mudar os valores de peso dos atributos? (s/n): ").lower()
        if mudar_pesos == 's':
            modificar_pesos()

        # Obtém o caso novo do usuário
        caso_novo = obter_entrada_usuario()

        # Calcula similaridades
        resultados = []
        for idx, caso_base in base_casos.iterrows():
            similaridade = similaridade_global(caso_novo, caso_base.to_dict(), PESOS)
            resultados.append((idx, similaridade, caso_base.to_dict()))

        # Ordena por similaridade (decrescente)
        resultados.sort(key=lambda x: x[1], reverse=True)
        mostrar_resultados(caso_novo, resultados)

        # Pergunta se quer salvar o caso
        salvar = input("\nDeseja salvar este caso na base? (s/n): ").lower()
        if salvar == 's':
            novo_id = salvar_novo_caso(caso_novo, base_casos, resultados)
            if novo_id:
                # Recarrega a base atualizada
                base_casos = carregar_base_real()

        # Pergunta se quer fazer nova consulta
        nova_consulta = input("\nDeseja fazer nova consulta? (s/n): ").lower()
        if nova_consulta != 's':
            print("\nFinalizando sistema RBC")
            break

# Execução do sistema
if __name__ == "__main__":
    main()